                                                           1. Importação de bibliotecas

In [52]:
import pandas as pd
import requests
import numpy as np
import time

                                                           2. Leitura de dados de entrada

In [ ]:
# 1. Carrega o CSV com os dados dos clientes
clientes = pd.DataFrame(pd.read_csv(r'/mnt/e/via_cep_etl_project/via_cep_etl_project/data/dados_cep_50.csv',delimiter=','))
clientes.head(10)

,nome,cep
0,João Silva,01001-000
1,Maria Souza,30140-110
2,Carlos Pereira,20040-010
3,Ana Oliveira,70040-010
4,Paulo Lima,59010-000
5,Fernanda Rocha,88015-701
6,Lucas Mendes,40020-000
7,Juliana Costa,69005-070
8,Ricardo Santos,69060-001
9,Larissa Almeida,60810-270


In [ ]:
# 2. Define a função para consultar a API ViaCEP
def consultar_cep(cep):
    url = f"https://viacep.com.br/ws/{cep}/json/"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            return None
    except Exception as e:
        print(f"Erro ao consultar o CEP {cep}: {e}")
        return None
    

In [ ]:
enderecos = []# 3. Cria uma lista para armazenar os dados de endereço
# 4. Consulta cada CEP e preenche os dados de endereço
for cep in clientes['cep']:
    dados = consultar_cep(cep)
    if dados and not dados.get('erro'):
        enderecos.append({
            'cep': cep,
            'logradouro': dados.get('logradouro'),
            'bairro': dados.get('bairro'),
            'cidade': dados.get('localidade'),
            'uf': dados.get('uf')
        })
    else:
        enderecos.append({
            'cep': cep,
            'logradouro': None,
            'bairro': None,
            'cidade': None,
            'uf': None
        })
    time.sleep(1)  # Respeitar limite de requisições da API gratuita


In [73]:
# 5. Cria um DataFrame com os endereços coletados
df_enderecos = pd.DataFrame(enderecos)

# 6. Substitui valores None por NaN
df_enderecos = df_enderecos.replace({None: np.nan})

# 7. Preenche valores ausentes com 'Não encontrado'
df_enderecos['logradouro'] = df_enderecos['logradouro'].fillna('Não encontrado')
df_enderecos['bairro'] = df_enderecos['bairro'].fillna('Não encontrado')
df_enderecos['cidade'] = df_enderecos['cidade'].fillna('Não encontrado')
df_enderecos['uf'] = df_enderecos['uf'].fillna('Não encontrado')


In [74]:

df_final = pd.merge(clientes, df_enderecos, on='cep')# 8. Junta os dados originais com os dados de endereço
df_final.to_csv('../output/clientes_consolidados.csv', index=False)# 9. Exporta o resultado final para CSV
df_final.head(7)  # 10. Exibe as primeiras linhas do DataFrame final

,nome,cep,logradouro,bairro,cidade,uf
0,João Silva,01001-000,Praça da Sé,Sé,São Paulo,SP
1,Maria Souza,30140-110,Rua Santa Rita Durão,Funcionários,Belo Horizonte,MG
2,Carlos Pereira,20040-010,Avenida Presidente Vargas,Centro,Rio de Janeiro,RJ
3,Ana Oliveira,70040-010,Quadra SBN Quadra 1,Asa Norte,Brasília,DF
4,Paulo Lima,59010-000,Avenida Presidente Café Filho,Praia do Meio,Natal,RN
5,Fernanda Rocha,88015-701,Avenida Jornalista Rubens de Arruda Ramos,Centro,Florianópolis,SC
6,Lucas Mendes,40020-000,Não encontrado,Não encontrado,Não encontrado,Não encontrado
